In [1]:
# Install required packages (run once in the notebook)
%pip install crewai langchain-groq

from crewai import Crew, Agent, Task
from langchain_groq import ChatGroq

Note: you may need to restart the kernel to use updated packages.


c:\ConferenceBot\ConferenceBot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 1. Configure LLM
# -----------------------------
llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0.3)

In [3]:
class WriterTask(Task):
    # Input field
    text: str

    # Required CrewAI fields
    description: str = "Generate a well-structured academic paragraph from input text"
    expected_output: str = "A polished academic paragraph based on input text"

    def __call__(self):
        prompt = f"""
Write a clear, coherent, and well-structured academic paragraph based on the following text:
{self.text}

Ensure formal, logical flow. Use scholarly tone, proper grammar, and connect ideas naturally. Do NOT use bullet points in output.
"""
        response = llm.invoke(prompt)
        paragraph = response.content if hasattr(response, "content") else response
        return {"paragraph": paragraph}

In [4]:
# 3. PolisherTask: refine paragraph
# -----------------------------
class PolisherTask(Task):
    draft: str

    description: str = "Polish an academic paragraph for tone, grammar, clarity, and flow"
    expected_output: str = "A polished, publication-ready paragraph"

    def __call__(self):
        prompt = f"""
Act as a professional academic editor. Polish the following paragraph for academic clarity, tone, grammar, and readability.
Ensure formal, cohesive, and publication-ready style.

Draft:
{self.draft}
"""
        response = llm.invoke(prompt)
        polished = response.content if hasattr(response, "content") else response
        return {"polished": polished}


In [5]:
# 4. Example Usage

input_text = """
Climate change is a pressing global issue. It leads to rising temperatures, melting ice caps, extreme weather events,
and has a significant impact on biodiversity. Human activities such as deforestation and carbon emissions are major contributors.
"""

# Instantiate tasks by passing field names
writer_task = WriterTask(text=input_text)
writer_result = writer_task()
draft_paragraph = writer_result["paragraph"]

polisher_task = PolisherTask(draft=draft_paragraph)
polisher_result = polisher_task()
polished_paragraph = polisher_result["polished"]


# 5. Display Results
# -----------------------------
print("Generated Paragraph:\n", draft_paragraph)
print("\nPolished Paragraph:\n", polished_paragraph)

Generated Paragraph:
 Climate change is a paramount global concern that necessitates immediate attention, as it precipitates a myriad of far-reaching consequences. The phenomenon is characterized by rising temperatures, which in turn trigger the melting of ice caps, thereby contributing to the escalation of extreme weather events. Furthermore, the impact of climate change on biodiversity is profound, with many species facing the threat of extinction due to the disruption of their habitats. It is essential to acknowledge that human activities play a pivotal role in exacerbating this issue, with deforestation and carbon emissions being major contributors to the exacerbation of climate change. The correlation between these human-induced factors and the worsening of climate change underscores the need for a concerted effort to mitigate the effects of global warming, thereby ensuring the long-term sustainability of the planet's ecosystems.

Polished Paragraph:
 Here is the polished paragrap